In [1]:
#テストデータ作成
import pandas as pd
# Create a sample DataFrame
data = {
    'id': [1,2,3],
    'first name': ['Suzuki', 'Sato', 'Takahashi']
}

df = pd.DataFrame(data)
df

,id,first name
0,1,Suzuki
1,2,Sato
2,3,Takahashi


https://api.dataplatform.cloud.ibm.com/v2/data_flows/doc/dataasset_and_connection_properties.html#dashdb

In [2]:
#DB書出し
import itc_utils.flight_service as itcfs
import pyarrow as pa

# create an instance of a flight client
flightClient = itcfs.get_flight_client()

flight_data_request = {
    'connection_name': """DB2WHOC_HDM""",   # name of the connection
    #'commit_frequency':  5000 optional parameter to trigger a commit every n records
    'interaction_properties': {
        'schema_name': 'HKWD', 
        'table_name': 'FLIGHTTEST',
        'table_action': 'replace' # truncate|append
        #'create_statement' : 'CREATE TABLE ...' # if you want to control data types
    },
}



# create a flight descriptor specifiying the data source or target
# content and structure of cmd are specific to IBM CP4D's flight service
flightDescriptor = itcfs.get_flight_descriptor(nb_data_request = flight_data_request)

# obtain a flight write stream
schema = pa.Schema.from_pandas(df, preserve_index=False)
writer, reader = flightClient.do_put(flightDescriptor, schema)

# write data to a data target
writer.write_table(pa.Table.from_pandas(df, schema))
writer.close()


In [3]:
#DB読み出し
flight_data_request = {
    'connection_name': """DB2WHOC_HDM""",
    'interaction_properties': {
        'schema_name': 'HKWD', 
        'table_name': 'FLIGHTTEST'
        #'select_statement': 'SELECT * FROM "HKWD"."FLIGHTTEST" FETCH FIRST 5000 ROWS ONLY'
    }
}

flightInfo = itcfs.get_flight_info(flightClient, nb_data_request=flight_data_request)

data_df_1 = itcfs.read_pandas_and_concat(flightClient, flightInfo, timeout=240)
data_df_1


,id,first name
0,1,Suzuki
1,2,Sato
2,3,Takahashi
